In [2]:
!pip install pandas sentence-transformers torch pyarrow --quiet


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [ ]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
import torch

ModuleNotFoundError: No module named 'pandas'

In [3]:
df = pd.read_parquet('/tf/dev/data/yugioh_cards.parquet')

In [5]:
df['type'].unique()

array(['Spell Card', 'Effect Monster', 'Normal Monster',
       'Flip Effect Monster', 'Link Monster', 'Trap Card',
       'Union Effect Monster', 'Fusion Monster',
       'Pendulum Effect Monster', 'XYZ Monster', 'Synchro Monster',
       'Synchro Tuner Monster', 'Tuner Monster', 'Gemini Monster',
       'Normal Tuner Monster', 'Spirit Monster', 'Ritual Effect Monster',
       'Skill Card', 'Token', 'Pendulum Effect Fusion Monster',
       'Ritual Monster', 'Toon Monster', 'Pendulum Normal Monster',
       'Synchro Pendulum Effect Monster', 'Pendulum Tuner Effect Monster',
       'XYZ Pendulum Effect Monster', 'Pendulum Effect Ritual Monster',
       'Pendulum Flip Effect Monster', 'Flip Tuner Effect Monster'],
      dtype=object)

In [11]:
def card_to_text(row):
    return f"{row['name']} - {row['type']} - {row.get('race', '')} - ATK {row.get('atk', '')} DEF {row.get('def', '')} - {row['desc']}"
    
df['search_text'] = df.apply(card_to_text, axis=1)
df["search_text"].iloc[0]

'"A" Cell Breeding Device - Spell Card - Continuous - ATK nan DEF nan - During each of your Standby Phases, put 1 A-Counter on 1 face-up monster your opponent controls.'

In [7]:
model = SentenceTransformer('all-MiniLM-L6-v2') 

/usr/local/lib/python3.11/dist-packages/torch/cuda/__init__.py:174: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 34: CUDA driver is a stub library (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [13]:
embeddings = model.encode(df['search_text'].tolist(), show_progress_bar=True)

Batches:   0%|          | 0/429 [00:00<?, ?it/s]

In [8]:
import faiss
import numpy as np


In [ ]:

embedding_dim = len(embeddings[0])
index = faiss.IndexFlatL2(embedding_dim)
index.add(np.array(embeddings).astype('float32'))

In [9]:
index = faiss.read_index('/tf/dev/data/yugioh.index')

In [ ]:
def search(query, k=5):
    query_vec = model.encode([query])
    D, I = index.search(np.array(query_vec).astype('float32'), k)
    return df.iloc[I[0]][['name',]]

In [24]:
search("Supreme King", k=10)

,name
11543,Supreme Rage
10915,Soul of the Supreme Celestial King
11367,Succubus Knight
10916,Soul of the Supreme King
11942,The Furious Sea King
9339,Protector of the Throne
11538,Supreme King Gate Magician
11541,Supreme King Z-ARC - Synchro Universe
6446,King's Knight
7780,Miracle of the Supreme King


In [ ]:
faiss.write_index(index, "dev/data/yugioh.index")